<a href="https://colab.research.google.com/github/nxxk23/kidney/blob/main/impute_missforest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install missingpy
!pip install scikit-learn==1.1.2

In [2]:
import sklearn
print(sklearn.__version__)

1.1.2


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/kidney/cut_outlier.csv')
df = df.drop(['Unnamed: 0'], axis=1)
df.head()

,PatientUID,PatientVisitUID,CWhen,eGFR,datetime,PreviousDateTime,Sex,Age,ENTY,NATNL,...,G4M2_Febuxostat,G3M2_Ezetimibe,G1M4_Non-dihydropyridine_CCB,G5M1_Ketoanalog_amino_acid,G2M7_GLP1_RA,G4M3_Uricosuric_agent,LabLocation,stage,stage_latest,Fold
0,2299,3386255,2559-08-15 09:51:54.913,71.7,2016-08-15 09:51:54.913,NaN,หญิง,73.0,Out Patient,ไทย,...,0.0,0.0,0.0,0.0,0.0,0.0,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2,2,0
1,2299,3640924,2559-11-11 09:40:06.053,66.8,2016-11-11 09:40:06.053,2016-08-15 09:51:54.913,หญิง,73.0,Out Patient,ไทย,...,0.0,0.0,0.0,0.0,0.0,0.0,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2,2,0
2,2299,4020970,2560-03-31 10:18:54.363,67.2,2017-03-31 10:18:54.363,2016-11-11 09:40:06.053,หญิง,74.0,Out Patient,ไทย,...,0.0,0.0,0.0,0.0,0.0,0.0,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2,2,0
3,2299,4286612,2560-07-05 10:40:40.763,69.2,2017-07-05 10:40:40.763,2017-03-31 10:18:54.363,หญิง,74.0,Out Patient,ไทย,...,0.0,0.0,0.0,0.0,0.0,0.0,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2,2,0
4,2299,4546471,2560-10-03 09:19:54.710,63.6,2017-10-03 09:19:54.710,2017-07-05 10:40:40.763,หญิง,74.0,Out Patient,ไทย,...,0.0,0.0,0.0,0.0,0.0,0.0,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2,2,0


In [5]:
data = pd.DataFrame(df)

In [6]:
import numpy as np

In [7]:
columns_to_missfr = [col for col in data.columns
                     if (col not in ['PatientUID', 'PatientVisitUID','eGFR',
                                     'InPatient', 'OutPatient', 'Emergency','ENTY',
                                     'VisitLocation', 'StartDTTM','EndDttm',
                                     'CWhen','datetime','PreviousDateTime'])]
data_selected = data[columns_to_missfr].copy()

In [8]:
print(data_selected.columns.tolist())

['Sex', 'Age', 'NATNL', 'BMI', 'BSA', 'Diastolic BP', 'Systolic BP', 'L1_eGFR', 'L10_Phosphorus', 'L11_Albumin', 'L12_Hemoglobin', 'L13_Hematocrit', 'L14_WBC', 'L15_Platelet', 'L16_TIBC', 'L18_Ferritin', 'L19_Blood_sugar', 'L2_BUN', 'L20_HbA1c', 'L21_Cholesterol', 'L22_Triglyceride', 'L23_LDL_Cholesterol', 'L24_HDL_Cholesterol', 'L25_Parathyroid_hormone', 'L26_Vitamin_D', 'L27_UrineAlbumin/Cr_ratio', 'L28_UrineProtein/Cr_ratio', 'L29_UrineVolume_24hrs', 'L3_Creatinine', 'L30_UrineUrea_24hrs', 'L32_UrineProtein_24hrs', 'L4_Sodium', 'L5_Potassium', 'L6_Chloride', 'L7_Bicarbonate', 'L8_Uric', 'L9_Calcium', 'D1_Hypertension', 'D10_Ischemic_heart_disease', 'D11_Heart_failure', 'D12_Thalassemia', 'D13_Kidney_stone', 'D14_Valvular_heart_disease', 'D15_Solid_malignancy', 'D16_Hematologic_malignancy', 'D2_Diabetes_mellitus', 'D3_Dyslipidemia', 'D4_SLE', 'D5_Systemic_sclerosis', 'D6_Rheumatoid_arthritis', 'D7_HIV_infection', 'D8_HBV_infection', 'D9_HCV_infection', 'G1M2_ARB', 'G3M1_Statin', 'G5M

In [9]:
for col in data_selected.columns:
  print(col, "\t", data_selected[col].dtypes, "\t", np.sum(data_selected[col].isna()))

Sex 	 object 	 0
Age 	 float64 	 0
NATNL 	 object 	 0
BMI 	 float64 	 92074
BSA 	 float64 	 91655
Diastolic BP 	 float64 	 39392
Systolic BP 	 float64 	 40188
L1_eGFR 	 float64 	 0
L10_Phosphorus 	 float64 	 255864
L11_Albumin 	 float64 	 168910
L12_Hemoglobin 	 float64 	 165642
L13_Hematocrit 	 float64 	 118627
L14_WBC 	 float64 	 119300
L15_Platelet 	 float64 	 119637
L16_TIBC 	 float64 	 312785
L18_Ferritin 	 float64 	 306794
L19_Blood_sugar 	 float64 	 192515
L2_BUN 	 float64 	 304389
L20_HbA1c 	 float64 	 311670
L21_Cholesterol 	 float64 	 49185
L22_Triglyceride 	 float64 	 305988
L23_LDL_Cholesterol 	 float64 	 169911
L24_HDL_Cholesterol 	 float64 	 310986
L25_Parathyroid_hormone 	 float64 	 318674
L26_Vitamin_D 	 float64 	 318670
L27_UrineAlbumin/Cr_ratio 	 float64 	 318674
L28_UrineProtein/Cr_ratio 	 float64 	 307383
L29_UrineVolume_24hrs 	 float64 	 318673
L3_Creatinine 	 float64 	 40
L30_UrineUrea_24hrs 	 float64 	 314958
L32_UrineProtein_24hrs 	 float64 	 308874
L4_Sodium 	 

In [10]:
import re
original_columns = data_selected.columns.tolist()
data_selected = data_selected.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '-', x))
data_selected.head()

,Sex,Age,NATNL,BMI,BSA,Diastolic-BP,Systolic-BP,L1_eGFR,L10_Phosphorus,L11_Albumin,...,G4M2_Febuxostat,G3M2_Ezetimibe,G1M4_Non-dihydropyridine_CCB,G5M1_Ketoanalog_amino_acid,G2M7_GLP1_RA,G4M3_Uricosuric_agent,LabLocation,stage,stage_latest,Fold
0,หญิง,73.0,ไทย,20.80,1.39,75.0,127.0,71.7,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2,2,0
1,หญิง,73.0,ไทย,20.00,1.37,70.0,132.0,66.8,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2,2,0
2,หญิง,74.0,ไทย,28.62,1.60,79.0,127.0,67.2,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2,2,0
3,หญิง,74.0,ไทย,21.94,1.41,82.0,129.0,69.2,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2,2,0
4,หญิง,74.0,ไทย,20.27,1.38,64.0,105.0,63.6,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,ห้องปฏิบัติการจุลทรรศน์วินิจฉัย,2,2,0


In [11]:
data_selected['Sex'] = data_selected['Sex'].replace({'ชาย': 0, 'หญิง': 1})
data_selected['Sex'].value_counts()

Sex
0    171458
1    147217
Name: count, dtype: int64

In [12]:
nationality_map = {'ไทย': 0}

caucasian = ['บริติช  (อังกฤษ, สก็อตแลนด์)', 'อเมริกัน', 'เยอรมัน', 'โอมาน', 'ออสเตรเลีย', 'แคนาดา',
             'อิตาลี', 'สวิส', 'ซีเรีย', 'เดนมาร์ก', 'นอร์เวย์', 'ดัตช์', 'สเปน', 'เบลเยียม', 'ไอซ์แลนด์',
             'ฝรั่งเศส', 'เซนต์คิตส์และเนวิส', 'สวีเดน', 'ออสเตรีย', 'ไซปรัส']
asian = ['ลาว','จีน','เวียดนาม','เขมรอพยพ','ไทยลื้อ','ญี่ปุ่น', 'จีน(ไต้หวัน)', 'พม่า']
other = ['ไม่ระบุ','อื่นๆ','ไร้สัญชาติ']

for nationality in asian:
    nationality_map[nationality] = 0

for nationality in caucasian:
    nationality_map[nationality] = 1

for nationality in other:
    nationality_map[nationality] = 0

data_selected['NATNL'] = data_selected['NATNL'].replace(nationality_map)
data_selected['NATNL'].value_counts()

NATNL
0    317607
1      1068
Name: count, dtype: int64

In [13]:
data_selected['LabLocation'] = np.where(data_selected['LabLocation'] == "ห้องปฏิบัติการจุลทรรศน์วินิจฉัย", 1, 0)
data_selected['LabLocation'].value_counts()

LabLocation
1    174284
0    144391
Name: count, dtype: int64

In [14]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

**using TPU**

    max_iter, defaults to 10.
    It taskes 40 mins for 5 iterations. so, it should complete at 1.20 hrs.
    and it stopped at 9 so it takes 1 hr

## **impute**

In [15]:
imputer = MissForest(random_state=0, verbose=1)
X = data_selected.copy()
X_imputed = imputer.fit_transform(X)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of 100 | elapsed:   10.4s remaining:  1.6min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   15.8s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 ou

Iteration: 0


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of 100 | elapsed:   19.1s remaining:  2.9min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   29.8s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.3s
[Parallel(n_jobs=96)]: Done 100 ou

Iteration: 1


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of 100 | elapsed:   19.9s remaining:  3.0min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   30.5s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.3s
[Parallel(n_jobs=96)]: Done 100 ou

...

Iteration: 7


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of 100 | elapsed:   19.4s remaining:  2.9min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   30.0s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 ou

Iteration: 8


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of 100 | elapsed:   19.4s remaining:  2.9min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   30.0s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=96)]: Done 100 ou

Iteration: 9


In [16]:
data_missfr = pd.DataFrame(X_imputed, columns=X.columns)
data_missfr

,Sex,Age,NATNL,BMI,BSA,Diastolic-BP,Systolic-BP,L1_eGFR,L10_Phosphorus,L11_Albumin,...,G4M2_Febuxostat,G3M2_Ezetimibe,G1M4_Non-dihydropyridine_CCB,G5M1_Ketoanalog_amino_acid,G2M7_GLP1_RA,G4M3_Uricosuric_agent,LabLocation,stage,stage_latest,Fold
0,1.0,73.0,0.0,20.8000,1.3900,75.0,127.00,71.70,2.497,3.874,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0
1,1.0,73.0,0.0,20.0000,1.3700,70.0,132.00,66.80,2.308,4.356,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0
2,1.0,74.0,0.0,28.6200,1.6000,79.0,127.00,67.20,2.172,4.119,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0
3,1.0,74.0,0.0,21.9400,1.4100,82.0,129.00,69.20,1.853,3.520,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0
4,1.0,74.0,0.0,20.2700,1.3800,64.0,105.00,63.60,2.592,4.526,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318670,1.0,74.0,0.0,27.3068,1.7356,85.0,166.00,72.86,2.100,4.200,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0
318671,1.0,74.0,0.0,24.1400,1.5600,64.0,129.00,68.69,2.000,3.900,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0
318672,1.0,74.0,0.0,25.0800,1.5600,65.0,120.00,45.08,2.097,4.100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0
318673,0.0,67.0,0.0,23.4847,1.7249,82.0,155.00,94.41,2.300,3.800,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,1.0


In [17]:
import numpy as np
columns_to_keep  = df[['PatientUID', 'PatientVisitUID','eGFR',
                       'InPatient', 'OutPatient', 'Emergency','ENTY',
                       'VisitLocation', 'StartDTTM','EndDttm',
                       'CWhen','datetime','PreviousDateTime']]
result = pd.concat([columns_to_keep, data_missfr], axis=1)
result.head()

,PatientUID,PatientVisitUID,eGFR,InPatient,OutPatient,Emergency,ENTY,VisitLocation,StartDTTM,EndDttm,...,G4M2_Febuxostat,G3M2_Ezetimibe,G1M4_Non-dihydropyridine_CCB,G5M1_Ketoanalog_amino_acid,G2M7_GLP1_RA,G4M3_Uricosuric_agent,LabLocation,stage,stage_latest,Fold
0,2299,3386255,71.7,0.0,1.0,0.0,Out Patient,เวชปฏิบัติทั่วไป (OPD)-D03,2559-08-15 08:13:26.043,2559-08-16 05:01:01.473,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0
1,2299,3640924,66.8,0.0,1.0,0.0,Out Patient,เวชปฏิบัติทั่วไป (OPD)-D03,2559-11-11 07:57:06.370,2559-11-12 05:01:01.903,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0
2,2299,4020970,67.2,0.0,1.0,0.0,Out Patient,ออร์โธปิดิกส์ (OPD)-D09,2560-03-31 08:13:19.123,2560-04-01 05:01:01.793,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0
3,2299,4286612,69.2,0.0,1.0,0.0,Out Patient,เวชปฏิบัติทั่วไป (OPD)-D03,2560-07-05 08:24:01.940,2560-07-06 05:01:02.133,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0
4,2299,4546471,63.6,0.0,1.0,0.0,Out Patient,เวชปฏิบัติทั่วไป (OPD)-D03,2560-10-03 07:37:22.777,2560-10-04 05:01:01.903,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0


In [18]:
columns_with_null  = result.isnull().any()
if columns_with_null.any():
    print("There are columns with null values.")
else:
    print("No columns have null values.")

There are columns with null values.


In [19]:
result.to_csv('/content/drive/MyDrive/kidney/MissForest.csv')